In [17]:
import pandas as pd

In [13]:
# !wget https://files.grouplens.org/datasets/movielens/ml-20m.zip

In [14]:
# !unzip ml-20m.zip

In [3]:
m = pd.read_csv('../data/ml-20m/movies.csv')
m = m[m.movieId.notnull()].reindex()
m['itemid']=m.movieId.apply(lambda x: str(int(x)))
m['product_name'] = m['title']
items = m[['itemid','product_name','genres']]
# items.to_json('items.json')
items

,itemid,product_name,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [41]:
items.to_json('../data/ml-20m/processed/items.json')

### Purchases and grouped purchases.

In [4]:
interactions = pd.read_csv('../data/ml-20m/ratings.csv')
interactions = interactions[interactions.rating>=4.0]
interactions = interactions.sort_values(['userId','timestamp']) 
interactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9995410 entries, 23 to 20000242
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 381.3 MB


In [5]:
interactions['itemid'] = interactions['movieId'].apply(str)
interactions['userid'] = interactions['userId'].apply(str)
interactions['amount'] = 1
interactions['date'] = interactions['timestamp']
interactions[['itemid', 'userid','amount','date']]
interactions

,userId,movieId,rating,timestamp,itemid,userid,amount,date
23,1,1079,4.0,1094785665,1079,1,1,1094785665
96,1,2959,4.0,1094785698,2959,1,1,1094785698
111,1,3996,4.0,1094785727,3996,1,1,1094785727
6,1,151,4.0,1094785734,151,1,1,1094785734
58,1,1374,4.0,1094785746,1374,1,1,1094785746
...,...,...,...,...,...,...,...,...
19999942,138493,823,4.5,1260209589,823,138493,1,1260209589
20000257,138493,68319,4.5,1260209720,68319,138493,1,1260209720
20000213,138493,40819,4.5,1260209726,40819,138493,1,1260209726
20000200,138493,33794,4.0,1260209811,33794,138493,1,1260209811


In [44]:
interactions.to_json('../data/ml-20m/processed/purchases.json')

In [6]:
interactions['itemids'] = interactions[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x : ','.join(x))
iii = interactions[['userId','itemids']].drop_duplicates()
iii

,userId,itemids
23,1,"1079,2959,3996,151,1374,2291,4306,1214,1278,12..."
176,2,"62,1210,1356,1974,2951,110,589,1259,2948,3513,..."
257,3,"589,1721,2118,2710,2722,2448,2505,2694,858,122..."
424,4,"10,356,454,480,589,377,586,350,368,370,594,520..."
455,5,"62,141,736,780,671,832,1042,1393,150,590,380,4..."
...,...,...
19999614,138489,"1275,1747,3671,1061,2021,29,318,858,50,2019,91..."
19999644,138490,"111,3217,3654,3897,1535,2589,199,277,1735,1280..."
19999790,138491,"1093,4128,3186,2857,6874"
19999841,138492,"2115,908,58,2599,1500,1358,1288,1732,1252,1234..."


In [46]:
iii.to_json('../data/ml-20m/processed/purchases_txt.json')

In [7]:
purchases=pd.read_json('../data/ml-20m/processed/purchases.json')
purchases['userid'] = purchases.userid.apply(str)
purchases['itemid'] = purchases.itemid.apply(str)
purchases_item_counts = purchases[['userid','itemid']]
purchases_user_counts = purchases[['userid','itemid']]
purchases_user_count = purchases.groupby(['userid']).size().to_frame('nr_of_purchases').reset_index()
purchases_user_count = purchases_user_count.sort_values(by=['nr_of_purchases'], ascending=False)
pu5=purchases_user_count[purchases_user_count.nr_of_purchases>=5]
purchases_pu5 = purchases[purchases.userid.isin(pu5.userid)]
purchases_item_count_pu5 = purchases_pu5.groupby(['itemid']).size().to_frame('nr_of_purchases').reset_index()
purchases_item_count_pu5 = purchases_item_count_pu5.sort_values(by=['nr_of_purchases'], ascending=False)
purchases_item_count_pu5

,itemid,nr_of_purchases
6620,318,55705
6186,296,52265
12160,593,50061
7545,356,47302
4894,260,42578
...,...,...
3318,131050,1
3317,131027,1
3316,131023,1
3313,130996,1


In [ ]:
purchases_pu5.to_json('../data/ml-20m/processed/purchases_pu5.json')

In [8]:
purchases_pu5['itemids'] = purchases_pu5[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = purchases_pu5[['userId','itemids']].drop_duplicates()
iii['userid']=iii['userId'].apply(str)
iii = iii[['userid','itemids']]


<ipython-input-8-dca95e343ced>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchases_pu5['itemids'] = purchases_pu5[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))


In [ ]:
iii.to_json('../data/ml-20m/processed/purchases_txt_pu5.json')

In [49]:
iii['userid'].to_frame().to_json('../data/ml-20m/processed/users_pu5.json')

In [50]:

items[items.itemid.isin(purchases_item_count_pu5.itemid)].to_json("../data/ml-20m/processed/items_pu5.json")

In [51]:
purchases_item_count_pu5.to_json("../data/ml-20m/processed/items_sorted_pu5.json")

In [52]:

pu5.to_json("../data/ml-20m/processed/users_sorted_pu5.json")

In [10]:

users = pd.read_json('../data/ml-20m/processed/users_pu5.json')
shuffled_users = users.sample(frac=1., random_state=42)
test_users = shuffled_users.iloc[:10000]
val_users = shuffled_users.iloc[10000:20000]
train_users = shuffled_users.iloc[20000:]

test_users.to_json("../data/ml-20m/processed/test_users.json")
val_users.to_json("../data/ml-20m/processed/val_users.json")
train_users.to_json("../data/ml-20m/processed/train_users.json") 

print("Total users with pu5", len(users))
len(train_users),len(val_users),len(test_users)



Total users with pu5 136677


(116677, 10000, 10000)

## Selecting a subset of users for model finding, 

In [19]:
users = pd.read_json('../data/ml-20m/processed/users_pu5.json')
few_shuffled_users = users.sample(frac=.2, random_state=42) # 27335
few_test_users = few_shuffled_users.iloc[:2500]
few_val_users = few_shuffled_users.iloc[2500:5000]
few_train_users = few_shuffled_users.iloc[5000:]

few_test_users.to_json("../data/ml-20m/small_processed/few_test_users.json")
few_val_users.to_json("../data/ml-20m/small_processed/few_val_users.json")
few_train_users.to_json("../data/ml-20m/small_processed/few_train_users.json") 

print("Total few users with pu5", len(few_shuffled_users))
len(few_train_users),len(few_val_users),len(few_test_users)

Total few users with pu5 27335


(22335, 2500, 2500)

In [54]:
!ls ../data/ml-20m/processed/*.json

../data/ml-20m/processed/items.json
../data/ml-20m/processed/items_pu5.json
../data/ml-20m/processed/items_sorted_pu5.json
../data/ml-20m/processed/purchases.json
../data/ml-20m/processed/purchases_pu5.json
../data/ml-20m/processed/purchases_txt.json
../data/ml-20m/processed/purchases_txt_pu5.json
../data/ml-20m/processed/test_users.json
../data/ml-20m/processed/train_users.json
../data/ml-20m/processed/users_pu5.json
../data/ml-20m/processed/users_sorted_pu5.json
../data/ml-20m/processed/val_users.json


In [61]:
# users_s = pd.read_json('../data/ml-1m/processed/users_pu5.json')
# shuffled_users_s = users_s.sample(frac=1., random_state=42)
# test_users_s = shuffled_users_s.iloc[:604]
# val_users_s = shuffled_users_s.iloc[604:1209]
# train_users_s = shuffled_users_s.iloc[1209:]

# test_users_s.to_json("../data/ml-1m/processed/test_users.json")
# val_users_s.to_json("../data/ml-1m/processed/val_users.json")
# train_users_s.to_json("../data/ml-1m/processed/train_users.json") 
# len(test_users_s),len(val_users_s), len(train_users_s)

(604, 605, 4825)